In [1]:
#
# importing the necessary libraries: 
# NumPy, SciPy (spatial algorithms), timing functions, a handler for csv files 
#
import numpy as np
import scipy.spatial as sp
import time
import csv

In [2]:
#
# Path to the pics storge directory
#
file_dir = "/Applications/SageMath/icp_init/pics/"

In [3]:
#
# Centering a point cloud A
#
def barycentered(A):
    #
    bar = np.sum(A, axis=1)/A.shape[1]
    #
    return A - bar[:, np.newaxis]

In [4]:
#
# Best orthogonal fit between two (centred) labelled point clouds
#
# A and B are assumed to have the same number of points
#
def best_fit(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na==nb
    #
    H = A @ B.T
    W, S, V = np.linalg.svd(H)
    #
    return V.T @ W.T

In [5]:
#
# Detecting nearest neighbours between two unlabelled point coluds A and B
#
# Assumption: cardinality of A <= cardinality of B 
#
# Input: point clouds A, B
#
# Output: for each point of A a matching point of B, as a 0/1-matrix
#
def nearest_neighbours(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na <= nb
    #
    tree = sp.KDTree(A.T, leafsize=10, compact_nodes=True, copy_data=True, balanced_tree=True)
    #
    matching = []
    I = matrix.identity(na)
    #
    for i in range(nb):
        _, ind = tree.query(B.T[i], k=1, p=2, workers=-1)
        matching += [I[ind]]
    #    
    return np.array(matching)

In [6]:
#
# ICP algorithm for two unlabelled point clouds A and B
#
# Assumption: cardinality of A <= cardinality of B
#
# Input:
# A, B = unlabelled point clouds
# init = initial orthogonal transformation
# max_iter = maximum number of iterations
# tol = tolerance for halting the computation
#
# Output:
# U = orthogonal transformation bringing A to B as close as possible
# nn = nearest neighbor matching between U*A and B
# dist = distance between U*A and B*nn (in the max singular value norm)
#
def icp(A, B, init=None, max_iter=100, tol=1e-16):
    #
    src = copy(A)
    dst = copy(B)
    #
    na = src.shape[1]
    nb = src.shape[1]
    #
    assert na <= nb
    #
    if (init is not None):
        src = init @ src
    #    
    prev_err = 0
    #
    for i in range(max_iter):
        #   
        nn = nearest_neighbours(src, dst)
        U = best_fit(src, dst @ nn)
        #    
        src = U @ src
        #
        diff = src - dst @ nn
        err  = np.linalg.norm(diff, 2)
        #
        if abs(prev_err - err) < tol:
            break
        #    
        prev_err = err
    #
    U = best_fit(A, src)
    nn = nearest_neighbours(U @ A, dst)
    #
    diff = U @ A - B @ nn
    dist = np.linalg.norm(diff, 2)
    #
    return U, nn, dist

In [7]:
#
# Image for testing a given point cloud P with *multiplicative* noise
# 
# Producing another point cloud Q0 = O * P * S, with O a random orthogonal transformation
# and S a random permutation. Then adding noise N with each entry being Gaussian with
# mean 0 and standard deviation sigma. Creating Q = Q0 (Hadamard product) N.
#
# Test: determine the distance dist_spec from the recovered image of P denoted by 
# R = o * P * S to the specimen Q0, and compare it to the distance between d_icp 
# between o * P * s and Q returned by the ICP algorithm.
#
# Here it is important to note that S is a permutation, while s is a matching of nearest 
# neighbours in o * P and Q. The latter does not have to be a permutation matrix. 
#
def img_point_cloud_mul_noise(P, sigma=0.05, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    normP = np.linalg.norm(P, 2)
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q0 = O @ P @ S
    N = np.random.normal(1.0, sigma, (dim, num))
    Q = Q0 * N
    diff = Q - Q0
    dist_noise = np.linalg.norm(diff, 2) / normP
    if verbose:
        print("Noise introduced (normalised): {}".format(dist_noise))
    #
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    sols = []
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        nn = nearest_neighbours(U @ P, Q)
        diff_init = U @ P - Q @ nn
        d_init = np.linalg.norm(diff_init, 2) / normP
        sols += [(U, nn, d_init)]
    sols = sorted(sols, key=lambda x: x[2])
    o_init, s_init, dist_init = sols[0]
    o, s, d = icp(P, Q, o_init)
    dist_icp = d / normP
    R = o @ P @ S
    diff = Q0 - R
    dist_spec = np.linalg.norm(diff, 2) / normP
    diff = o - O
    dist_ortho = np.linalg.norm(diff, 2)
    diff = s.T - S
    dist_per = np.linalg.norm(diff, 'fro')**2/(2*num)
    if verbose:
        print("Orthogonal transformation found:")
        print(o)
        print("Distance to the initial one?")
        print(dist_ortho)
        print("Nearest neighbour matching found:")
        print(s.T)
        print("Normalised Hamming distance to the initial permutation:")
        print(dist_per)
        print("Normalised distance to noisy image:")
        print(dist_icp)
        print("Actual distance to specimen (normalised):")
        print(dist_spec)
    #
    flag = ( dist_spec < dist_icp )
    #
    PointsQ0 = point3d(Q0.T, size=2, color='red')
    PointsQ = point3d(Q.T, size=2, color='blue')
    PointsR = point3d(R.T, size=2, color='green')
    img = PointsQ0 + PointsQ + PointsR
    p = img.plot()
    t = time.process_time()
    p.save(file_dir+"mul_noise_pic_{}.html".format(t))
    #p.show(viewer="jmol3")
    #
    return flag, dist_noise, dist_icp, dist_spec, dist_ortho, dist_per

In [8]:
#
# Image for testing a given point cloud P with *additive* noise
# 
# Producing another point cloud Q0 = O * P * S, with O a random orthogonal transformation
# and S a random permutation. Then adding noise N with each entry being Gaussian with
# mean 0 and standard deviation sigma. Creating Q = Q0 + N.
#
# Test: determine the distance dist_spec from the recovered image of P denoted by 
# R = o * P * S to the specimen Q0, and compare it to the distance between d_icp 
# between o * P * s and Q returned by the ICP algorithm.
#
# Here it is important to note that S is a permutation, while s is a matching of nearest 
# neighbours in o * P and Q. The latter does not have to be a permutation matrix. 
#
def img_point_cloud_add_noise(P, sigma=0.05, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    normP = np.linalg.norm(P, 2)
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q0 = O @ P @ S
    N = np.random.normal(0.0, sigma, (dim, num))
    Q = Q0 + N
    dist_noise = np.linalg.norm(N, 2) / normP
    if verbose:
        print("Noise introduced (normalised): {}".format(dist_noise))
    #
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    sols = []
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        nn = nearest_neighbours(U @ P, Q)
        diff_init = U @ P - Q @ nn
        dist_init = np.linalg.norm(diff_init, 2) / normP
        sols += [(U, nn, dist_init)]
    sols = sorted(sols, key=lambda x: x[2])
    o_init, s_init, d_init = sols[0]
    o, s, d = icp(P, Q, o_init)
    dist_icp = d / normP
    R = o @ P @ S
    diff = Q0 - R
    dist_spec = np.linalg.norm(diff, 2) / normP
    diff = o - O
    dist_ortho = np.linalg.norm(diff, 2)
    diff = s.T - S
    dist_per = np.linalg.norm(diff, 'fro')**2/(2*num)
    if verbose:
        print("Orthogonal transformation found:")
        print(o)
        print("Distance to the initial one?")
        print(dist_ortho)
        print("Nearest neighbour matching found:")
        print(s.T)
        print("Normalised Hamming distance to the initial permutation:")
        print(dist_per)
        print("Normalised distance to noisy image:")
        print(dist_icp)
        print("Actual distance to specimen (normalised):")
        print(dist_spec)
    #
    flag = ( dist_spec < dist_icp )
    #
    PointsQ0 = point3d(Q0.T, size=2, color='red')
    PointsQ = point3d(Q.T, size=2, color='blue')
    PointsR = point3d(R.T, size=2, color='green')
    img = PointsQ0 + PointsQ + PointsR
    p = img.plot()
    t = time.process_time()
    p.save(file_dir+"add_noise_pic_{}.html".format(t))
    #p.show(viewer="jmol3")
    #
    return flag, dist_noise, dist_icp, dist_spec, dist_ortho, dist_per

In [9]:
#
# Image for testing a given *unoccluded* point cloud P with different levels of occlusion
# 
# Producing another point cloud Q0 = O * P * S, with O a random orthogonal transformation
# and S a random permutation. Then adding some extra occlusion points X to Q0 in order to 
# obtain a new *occluded* point cloud Q. This is control by level variable: level = % of 
# added occlusion points, a positive real number.  
#
# Test: determine the distance dist_spec from the recovered image of P denoted by 
# R = o * P * S to the specimen Q0, and compare it to the distance between d_icp between 
# o * P * s and Q returned by the ICP algorithm.
#
# Here it is important to note that S is a permutation, while s is a matching of nearest 
# neighbours in o * P and Q. The latter does not have to be a permutation matrix. 
#
def img_point_cloud_occluded(P, level=0.03, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    occ = int(num * level)
    normP = np.linalg.norm(P, 2)
    if verbose:
        print("Number of genuine points:   {}".format(num))
        print("Number of occlusion points: {}".format(occ))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num+occ))
    #
    P = barycentered(P)
    #
    Q0 = O @ P
    X  = [np.random.uniform(np.min(P[i,:]), np.max(P[i,:]), occ) for i in range(dim)]
    X  = np.array(X)
    X  = O @ X
    dist_occ = np.linalg.norm(X, 2) / normP
    if verbose:
        print("Occlusion introduced (normalised): {}".format(dist_occ))
    #
    Q = np.c_[Q0, X]
    Q = Q @ S
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    sols = []
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        nn = nearest_neighbours(U @ P, Q)
        diff_init = U @ P - Q @ nn
        dist_init = np.linalg.norm(diff_init, 2) / normP
        sols += [(U, nn, dist_init)]
    sols = sorted(sols, key=lambda x: x[2])
    o_init, s_init, d_init = sols[0]
    o, s, d = icp(P, Q, o_init)
    dist_icp = d / normP
    R = o @ P
    diff = Q0 - R
    dist_spec = np.linalg.norm(diff, 2) / normP
    diff = o - O
    dist_ortho = np.linalg.norm(diff, 2)
    if verbose:
        print("Orthogonal transformation found:")
        print(o)
        print("Distance to the initial one?")
        print(dist_ortho)
        print("Normalised distance to occluded image:")
        print(dist_icp)
        print("Actual distance to specimen (normalised):")
        print(dist_spec)
    #
    flag = ( dist_spec < dist_icp )
    #
    PointsQ0 = point3d(Q0.T, size=2, color='red')
    PointsQ = point3d(Q.T, size=2, color='blue')
    PointsR = point3d(R.T, size=2, color='green')
    img = PointsQ0 + PointsQ + PointsR
    p = img.plot()
    t = time.process_time()
    p.save(file_dir+"occluded_pic_{}.html".format(t))
    #p.show(viewer="jmol3")
    #
    return flag, dist_occ, dist_icp, dist_spec, dist_ortho

In [10]:
#
# Red points -> specimen / initial cloud
# Blue points -> noisy / occluded cloud
# Green points -> recovered cloud / orthogonal transform from ICP applied to specimen
#
def create_pics(filename):
    #
    P = []
    #
    text = ['Succes?', 'Noise/occlusion introduced:', 'Distance to noisy/occluded image:',\
            'Distance to specimen:', 'Distance to initial orthogonal transformation:',\
            'Hamming distance to initial permutation:']
    #
    f = open(filename)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #    
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(filename))
    #
    print("*** Multiplicative noise ***")
    output = img_point_cloud_mul_noise(P, sigma=0.1, verbose=False)
    output = zip(text, output)
    for s in output:
        print(s[0], s[1])
    #
    print("*** Additive noise ***")
    output = img_point_cloud_add_noise(P, sigma=0.01, verbose=False)
    output = zip(text, output)
    for s in output:
        print(s[0], s[1])
    #
    print("*** Occluded image ***")
    output = img_point_cloud_occluded(P, level=0.4, verbose=False)
    output = zip(text, output)
    for s in output:
        print(s[0], s[1])
    #
    print("#"*42)

In [11]:
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
create_pics("Teapot.csv")

File read: Teapot.csv
*** Multiplicative noise ***
Succes? True
Noise/occlusion introduced: 0.09623278221424408
Distance to noisy/occluded image: 0.722060249419136
Distance to specimen: 0.002046640530933312
Distance to initial orthogonal transformation: 0.002339757192820692
Hamming distance to initial permutation: 0.2706552706552707
*** Additive noise ***
Succes? True
Noise/occlusion introduced: 0.07429662250473684
Distance to noisy/occluded image: 0.800320079443211
Distance to specimen: 0.007028410960609778
Distance to initial orthogonal transformation: 0.008251697856528437
Hamming distance to initial permutation: 0.4045584045584046
*** Occluded image ***
Succes? True
Noise/occlusion introduced: 0.674130384687683
Distance to noisy/occluded image: 1.0507670145316186
Distance to specimen: 0.01582788767116019
Distance to initial orthogonal transformation: 0.02016006013707214
##########################################


In [12]:
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
create_pics("Bunny.csv")

File read: Bunny.csv
*** Multiplicative noise ***
Succes? True
Noise/occlusion introduced: 0.08556490079772003
Distance to noisy/occluded image: 0.7634613285504815
Distance to specimen: 0.0020571270974800476
Distance to initial orthogonal transformation: 0.0020829409029825456
Hamming distance to initial permutation: 0.2821969696969697
*** Additive noise ***
Succes? True
Noise/occlusion introduced: 0.07384407731712267
Distance to noisy/occluded image: 0.7781196322806718
Distance to specimen: 0.0074610408887154575
Distance to initial orthogonal transformation: 0.010704750818847942
Hamming distance to initial permutation: 0.40340909090909094
*** Occluded image ***
Succes? True
Noise/occlusion introduced: 0.6699592068074357
Distance to noisy/occluded image: 1.178692678439183
Distance to specimen: 0.060442745849137704
Distance to initial orthogonal transformation: 0.06877082714598488
##########################################


In [13]:
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
create_pics("Cow.csv")

File read: Cow.csv
*** Multiplicative noise ***
Succes? True
Noise/occlusion introduced: 0.08754298317369599
Distance to noisy/occluded image: 0.9962748523690172
Distance to specimen: 0.006532911920128182
Distance to initial orthogonal transformation: 0.009075851783602904
Hamming distance to initial permutation: 0.5930232558139535
*** Additive noise ***
Succes? True
Noise/occlusion introduced: 0.05617425545810242
Distance to noisy/occluded image: 0.9271091856559135
Distance to specimen: 0.0032604618811217403
Distance to initial orthogonal transformation: 0.003393032759648465
Hamming distance to initial permutation: 0.574750830564784
*** Occluded image ***
Succes? True
Noise/occlusion introduced: 0.5197020070580833
Distance to noisy/occluded image: 0.9755758538377661
Distance to specimen: 0.006470808254403098
Distance to initial orthogonal transformation: 0.009717699551580262
##########################################
